In [1]:
import os
import platform
from google.cloud import bigquery
from google.cloud import storage

In [2]:
app_path = os.getcwd()
os.chdir(os.getcwd())
filesep = '\\' if platform.system() == 'Windows' else '/'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "creds" + filesep + "sarasmaster-524142bf5547.json"

In [3]:
#Pass bigquery.SourceFormat.NEWLINE_DELIMITED_JSON for JSON in delimiter type and bigquery.SourceFormat.CSV for CSV
def loadfiletobigquery(file_name, dataset_id, table_name, delimitertype, loadtype, skipheader):
    client = bigquery.Client()
    table_ref = client.dataset(dataset_id).table(table_name)
    job_config = bigquery.LoadJobConfig()
    if skipheader is not None:
        job_config.skip_leading_rows = skipheader
    job_config.source_format = delimitertype
    if delimitertype == bigquery.SourceFormat.CSV:
        job_config.autodetect = True
    job_config.write_disposition = loadtype

    load_job = client.load_table_from_uri(
        file_name,
        table_ref,
        job_config=job_config)  # API request

    assert load_job.job_type == 'load'

    load_job.result()  # Waits for table load to complete.

    assert load_job.state == 'DONE'

In [30]:
print(bigquery.WriteDisposition.WRITE_TRUNCATE)
print(bigquery.WriteDisposition.WRITE_APPEND)
print(bigquery.SourceFormat.NEWLINE_DELIMITED_JSON)
print(bigquery.SourceFormat.CSV)

WRITE_TRUNCATE
WRITE_APPEND
NEWLINE_DELIMITED_JSON
CSV


In [53]:
datasetid = 'kopari'
tablename = 'shopify_products'
client = bigquery.Client()
table_ref = client.dataset(datasetid).table(tablename)
print(table_ref)

TableReference('sarasmaster-201918', 'kopari', 'shopify_products')


In [57]:
filename = 'gs://sarasmaster/kopari/shopify/products/products_1.json'
datasetid = 'kopari'
tablename = 'shopify_products'
delimitertype = 'NEWLINE_DELIMITED_JSON'
loadtype = 'WRITE_TRUNCATE'
loadfiletobigquery(filename, datasetid, tablename, delimitertype, loadtype, None)

In [58]:
filename = 'gs://sarasmaster/sarasdata/client_shopify_entitlements.csv'
datasetid = 'sarasdata'
tablename = 'client_shopify_entitlements'
delimitertype = 'CSV'
loadtype = 'WRITE_TRUNCATE'
loadfiletobigquery(filename, datasetid, tablename, delimitertype, loadtype, 1)

In [5]:
job_config = bigquery.LoadJobConfig()
job_config.schema = [
    bigquery.SchemaField('name', 'STRING'),
    bigquery.SchemaField('post_abbr', 'STRING')
]

In [6]:
print(job_config.schema)

[SchemaField('name', 'STRING', 'NULLABLE', None, ()), SchemaField('post_abbr', 'STRING', 'NULLABLE', None, ())]
